In this notebook, I give an example of how to use a custom likelihood function. Use NBViewer to view the notebook [here](https://nbviewer.jupyter.org/github/as4529/gp3/blob/master/examples/lif.ipynb?flush_cache=true). See the accompanying file (lif.py) for the implementation of the likelihood. 

In [1]:
import sys
sys.path.insert(0, "..")
from gp3.inference import MFSVI, FullSVI
from gp3.utils import data as sim
from gp3.kernels.kernels import RBF
from gp3.utils.transforms import softplus, inv_softplus
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange, tqdm_notebook
from ipywidgets import IntProgress
import numpy as np
from lif import LIFLike, LIFSim

## Problem Overview and Data Simulation

**Short description of problem**: We want to infer the conditional probability of a neuron firing given stimulation at particular spatial locations. We let $s_i$ denote the "gain" of stimulation at a given location $x_i$. On the right, we assume that we stimulate the neuron at a set of spatial locations $\{x_i\}_{i=1}^N$, and we receive the timing of the first spikes $\{y_i\}_{i=1}^N$. This $y_i$ is some stochastic function of $s_i$. For intuition, note that where the gain of stimulation is higher, the neurons tend to fire sooner.

The inference problem is as follows: given the timings of spikes at a sub-grid of stimulation locations, we want to infer the overall cell shape and quantify our uncertainty.  For inference, we put a GP prior on $\log s$.

$$\log s \sim \mathcal{GP}(\mu(\cdot), K(\cdot, \cdot))$$
$$y_i \sim \text{LIF}(s(x_i))$$

where LIF denotes a leaky integrate and fire likelihood, where the only free parameter is $s$. 

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
f = sim.sim_f(X, RBF(20., 1.), mu = 5.) - 1e-3*np.sum(np.square(X-50), 1)# + np.random.normal(size=X.shape[0])
ls = np.ones(len(f))
current = f + np.random.normal(size=len(f))*1.5
lif_gen = LIFSim(l=ls)
spikes = lif_gen.sim(current)

X_part, y_part = sim.rand_partial_grid(X, spikes, 0.3)
X_full, y_full, obs_idx, imag_idx = sim.fill_grid(X_part, y_part)
current_part = current[obs_idx]
color = np.zeros(X_full.shape[0])
color[obs_idx] = 1.0

trace_func = go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2.5, color=color), name = "cell parameter")
trace_current = go.Scatter3d(x = X_part[:,0], y = X_part[:,1], z=current[obs_idx], mode = 'markers', marker=dict(size = 2.5,), name = "current")
trace_draws = go.Scatter3d(x = X_part[:,0], y = X_part[:,1], z=spikes[obs_idx], mode = 'markers', marker=dict(size = 2.5), name = "spike times")
fig = tools.make_subplots(rows=1, cols=3, specs=[[{'is_3d': True}, {'is_3d': True}, {'is_3d': True}]], horizontal_spacing = 0.)
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_current, 1, 2)
fig.append_trace(trace_draws, 1, 3)
fig['layout']['xaxis3'].update(title='xaxis 3 title', showgrid=False)
fig['layout']['yaxis3'].update(title='yaxis 3 title', showgrid=False)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]  [ (1,3) scene3 ]




**More comprehensive description of LIF** (feel free to skip): We use a simplified leaky integrate-and-fire model to characterize the response of a neuron's membrane potential to stimulation at a spatial location $x_i$. We assume

$$ \begin{cases}
 V(t^+)=V_{{\rm reset}} & {\rm if \  cell} {\rm \ spikes \  at \  time} \ t  \\
 \mathrm{d}V(t)=g [V_{{\rm resting}}- V(t)] + l_is(x_i) & {\rm otherwise} \end{cases}$$
where $V_{{\rm reset}}$ is the reset voltage, $V_{{\rm resting}}$ is the resting voltage, and $g$ is the membrane resistance. Here  $l_i s(x_i) $ is the intensity of stimulation that the cell received given a stimulation at $x_i$,  where $l_i$ is the power level of the stimulation and $s(\cdot)$ is the gain of stimulation in space. $s(\cdot)$ is the function we are interested in modeling. Given the membrane potential, we model the firing probability as 
$$ {\rm pr}(\mathrm{d} N(t)=1) =  \psi( V(t)-V_{{\rm th}}),$$

where $\psi$ is a sigmoid function, and $V_{{\rm th}}$ is the spiking threshold. In summary, the LIF model reduces to, assuming that cell $j$ has not fired before time $t$, 

$$ V(t) = \int_{0}^t l_i s(x_i-z) \exp\big(-g \cdot (t-u) \big) \mathrm{d}u $$

Thus, the probability of first spike at time $t$ is 	
$$\lambda_{i,j}(t) = \exp\left\{ \int_0^t \log\big[ 1- \psi( V_j^i(s)-V_{j,{\rm th}}) \big] \mathrm{d} s\right\} \psi\big( V_j^i(t)-V_{j,{\rm th}} \big)$$	

and the probability of no spikes up to time $t$ is  

$${\rm pr}(N_{j,i}(t)=0) =  1- \int_0^t \lambda_{i,j}(t) \mathrm{d} t = \exp\left\{ \int_0^t \log\big[ 1- \psi( V_j^i(s)-V_{j,{\rm th}}) \big] \mathrm{d} s\right\}$$

## Inference

For inference, we run gp3's mean-field stochastic variational inference. Note that we assume we have optimized the kernel parameters ahead of time, and we have a good estimate (online kernel parameter optimization is in the works).

In [3]:
mu = np.ones(X.shape[0])*5
inf_svi = MFSVI(X, y_part, [RBF(20., 1.),RBF(20., 1.)], LIFLike(ts = y_part, l=np.ones(len(y_part))), mu = mu, obs_idx = obs_idx)
inf_svi.max_grad = 10.
inf_svi.run(200)

From the SVI inference, we get an estimate of $s$, as well as the posterior variance. We could then use this posterior variance to select the next set of points to stimulate.

In [4]:
trace_svipred = go.Scatter3d(x = X[:,0], y = X[:,1], z=inf_svi.predict(), mode = 'markers', marker=dict(size = 2, color = color), name = "svi prediction")
trace_svivar = go.Scatter3d(x = X[:,0], y = X[:,1], z=np.exp(inf_svi.q_S), mode = 'markers', marker=dict(size = 2, color = color), name = "svi variances")
#trace_svivar = go.Scatter3d(x = X[:,0], y = X[:,1], z=np.diag(inf_svi.full_S()), mode = 'markers', marker=dict(size = 2, color = color), name = "svi variances")

fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_svipred, 1, 1)
fig.append_trace(trace_svivar, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



In [6]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=inf_svi.q_mu, mode = 'markers', marker=dict(size = 2, color = color), name = "svi variances")])